In [16]:
# | default_exp routes.jupyter

In [17]:
# | exporti

from dataclasses import dataclass, field
from typing import Union
import datetime as dt
import domolibrary.utils.DictDot as util_dd
from dateutil.parser import parse

import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.jupyter as jupyter_routes
# import domolibrary.client.DomoError as de
# import domolibrary.utils.chunk_execution as ce

[![Tutorial Video pt 4 of 4]](https://youtu.be/RbfbQ8V2erM)

## DomoJupyter Content and Class


In [18]:
#| export

@dataclass
class DomoJupyter_Content:
    name: str
    folder: str
    last_modified: dt.datetime
    file_type: str
    content: str

    auth: dmda.DomoJupyterAuth = field(repr=False)

    default_export_folder: str = "export"

    def __post_init__(self):

        dmda.test_is_jupyter_auth(self.auth)

        if self.folder.endswith(self.name):
            self.folder = self.folder.replace(self.name, "")

    @classmethod
    def _from_json(cls, obj: dict, auth: dmda.DomoJupyterAuth):
        dd = util_dd.DictDot(obj) if not isinstance(obj, util_dd.DictDot) else obj

        dc = cls(
            name=dd.name,
            folder=dd.path,
            last_modified=parse(dd.last_modified),
            file_type=dd.type,
            auth=auth,
            content=obj.get("content"),
        )

        return dc

    def export_content(
        self,
        output_folder: str = None,
        file_name: str = None,
    ):
        output_folder = output_folder or os.path.join(
            self.default_export_folder, self.folder
        )

        file_name = file_name or self.name

        if not os.path.exists(output_folder):
            print(output_folder)
            os.makedirs(output_folder)

        content_str = self.content
        if isinstance(self.content, dict):
            import json

            content_str = json.dumps(self.content)

        output_path = os.path.join(output_folder, file_name)
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(content_str)
            f.close()

        return output_path

    async def update(
        self,
        jupyter_folder: str = None,
        jupyter_file_name: str = None,
        debug_api: bool = False,
    ):
        if jupyter_folder and jupyter_file_name:
            content_patuh = f"{jupyter_folder}/{jupyter_file_name}"

        if len(self.folder) > 0:
            content_path = f"{self.folder}/{self.name}"

        else:
            content_path = self.name

            if content_path.lower().startswith(self.default_export_folder.lower()):
                content_path = content_path.replace(self.default_export_folder, "")

        content_path = "/".join(os.path.normpath(content_path).split(os.sep))

        return await jupyter_routes.update_jupyter_file(
            auth=self.auth,
            content_path=content_path,
            new_content=self.content,
            debug_api=debug_api,
            debug_num_stacks_to_drop = 2,
            parent_class = self.__class__.__name__
        )


In [19]:

#|export

@dataclass
class DomoJupyter:
    auth: dmda.DomoJupyterAuth = field(repr=False)
    content: [DomoJupyter_Content] = field(default=None)

    jupyter_token = None
    service_location = None
    service_prefix = None

    def __post_init__(self):
        dmda.test_is_jupyter_auth(self.auth)

        if hasattr(self.auth, "domo_password") and not isinstance(
            self.auth, dmda.DomoJupyterFullAuth
        ):
            self.auth = DomoJupyterFullAuth.convert_auth(
                full_auth=self.auth,
                jupyter_token=self.jupyter_token,
                service_location=self.service_location,
                service_prefix=self.service_prefix,
            )
        if hasattr(self.auth, "developer_token") and not isinstance(
            self.auth, DomoJupyterDeveloperToken
        ):
            

            self.auth = dmda.DomoJupyterTokenAuth.convert_auth(
                auth=self.auth,
                jupyter_token=self.jupyter_token,
                service_location=self.service_location,
                service_prefix=self.service_prefix,
            )

    async def get_content(
        self,
        debug_api: bool = False,
        return_raw: bool = False,
        is_recursive: bool = True,
        content_path: str = "",
    ):
        if is_recursive:
            res = await jupyter_routes.get_content_recursive(
                auth=self.auth, debug_api=False, content_path=content_path, debug_num_stacks_to_drop= 3, parent_class = self.__class__.__name__
            )
            content_ls = res.response

        else:
            res = await jupyter_routes.get_jupyter_content(
                auth=self.auth, debug_api=False, content_path=content_path, debug_num_stacks_to_drop =2, parent_class = self.__class__.__name__
            )

            content_ls = res.response["content"]

        if return_raw:
            return res

        return [
            DomoJupyter_Content._from_json(obj, auth=self.auth) for obj in content_ls
        ]

#### sample impelemntation of get_contet


In [20]:
#| eval: false

import os
import domolibrary.client.DomoAuth as dmda

jupyter_token = "mgaHHqXQafZhWMIcbT5cVsW9KZY7c4"
service_location = "jupyter-prod1.domodatascience.com"
service_prefix = "/user/domo-community-1893952720/19fb3535/"

dj_auth = dmda.DomoJupyterTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    jupyter_token=jupyter_token,
    service_location=service_location,
    service_prefix=service_prefix,
)

domo_dj = DomoJupyter(auth=dj_auth)

domo_dj_content = await domo_dj.get_content(
    debug_api=False, 
    return_raw=False, 
    is_recursive=True
)

[domo_content.name for domo_content in domo_dj_content]


['tutorial_env.ipynb',
 'my_terrible_demo.txt',
 'UpdateDatasets.ipynb',
 'untitled1.txt',
 'DataflowExecution.ipynb',
 'MakeDomoFaster.ipynb',
 'untitled1.txt',
 'get_content.ipynb',
 'Untitled.ipynb',
 'instance_pdp_access.ipynb',
 'README.md',
 'untitled.txt',
 'env.txt',
 'my_great_demo.txt',
 'datatypes.ipynb']

#### sample implementation of updating content with DomoJupyter_Content class


In [21]:
#|eval: False

import os
import domolibrary.client.DomoAuth as dmda

jupyter_token = "mgaHHqXQafZhWMIcbT5cVsW9KZY7c4"
service_location = "jupyter-prod1.domodatascience.com"
service_prefix = "/user/domo-community-1893952720/19fb3535/"

dj_auth = dmda.DomoJupyterTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    jupyter_token=jupyter_token,
    service_location=service_location,
    service_prefix=service_prefix,
)

domo_dj = DomoJupyter(auth=dj_auth)

domo_dj_content = await domo_dj.get_content(
    debug_api=False, 
    return_raw=False, 
    is_recursive=True
)

test_content = domo_dj_content[1]

test_content.content = "jae rocks at debugging on the fly"

await test_content.update(debug_api=False)

ResponseGetData(status=200, response={'name': 'untitled1.txt', 'path': 'untitled1.txt', 'last_modified': '2023-09-15T22:24:41.623000Z', 'created': '2023-09-15T22:24:41.623000Z', 'content': None, 'format': None, 'mimetype': 'text/plain', 'size': 33, 'writable': True, 'type': 'file'}, is_success=True, parent_class=None, traceback_details=TracebackDetails(function_name='update', file_name='/tmp/ipykernel_4655/1805607417.py', function_trail='<module> -> update', traceback_stack=[<FrameSummary file /tmp/ipykernel_4655/3813769592.py, line 30 in <module>>, <FrameSummary file /tmp/ipykernel_4655/1805607417.py, line 85 in update>], parent_class='DomoJupyter_Content'))

#### sample implementation of export_content

In [22]:
#| eval : False

import os
import domolibrary.client.DomoAuth as dmda

jupyter_token = "mgaHHqXQafZhWMIcbT5cVsW9KZY7c4"
service_location = "jupyter-prod1.domodatascience.com"
service_prefix = "/user/domo-community-1893952720/19fb3535/"

dj_auth = dmda.DomoJupyterTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    jupyter_token=jupyter_token,
    service_location=service_location,
    service_prefix=service_prefix,
)

domo_dj = DomoJupyter(auth=dj_auth)

domo_dj_content = await domo_dj.get_content(
    debug_api=False, 
    return_raw=False, 
    is_recursive=True
)

test_content = domo_dj_content[1]

test_content.content = "jae rocks at debugging on the fly"
test_content

await test_content.update(debug_api=False)

test_content.export_content(output_folder= '../test/')


'../test/untitled1.txt'

In [23]:
#| hide
import nbdev
nbdev.nbdev_export()